# Jobs, Sessions, and Batch Execution (EXAM CRITICAL!)

> **Exam Weight**: Part of 15% (Section 4) | **Must Master**: ✅✅

**📌 Note**: This notebook uses `FakeBackend` from `qiskit_ibm_runtime.fake_provider` for testing without real hardware. Session/Batch patterns are shown for reference but require real hardware to fully execute.

## Learning Objectives
By the end of this notebook, you will be able to:
- Understand the job lifecycle and different execution states
- Differentiate between `job.result()` (blocking) and `job.status()` (non-blocking)
- Compare Jobs, Batches, and Sessions for different use cases
- Implement proper job monitoring and retrieval patterns
- Know when to use Sessions for iterative algorithms (VQE/QAOA)

---

## 🧠 Conceptual Deep Dive

### The Restaurant Analogy 🍽️

Imagine a **busy restaurant** - this perfectly maps to IBM Quantum Runtime execution:

| Restaurant Concept | Quantum Equivalent | Key Behavior |
|-------------------|-------------------|--------------|
| **Single Order** | **Job** | One circuit submitted, waits in kitchen queue |
| **Group Order** | **Batch** | Multiple independent circuits, served together when all ready |
| **Private Room** | **Session** | Reserved dedicated access, sequential courses (iterative jobs) |

```
Restaurant Kitchen (Quantum Backend):
┌─────────────────────────────────────────────────────────────┐
│                                                             │
│  📝 Single Order (Job)                                      │
│     - Quick one-off experiments                             │
│     - Goes to general queue                                 │
│     - No guaranteed timing relationship                     │
│                                                             │
│  🍽️ Group Order (Batch)                                    │
│     - Parameter sweeps, independent circuits                │
│     - All submitted together, run in parallel               │
│     - Results returned as a group                           │
│                                                             │
│  🚪 Private Room (Session)                                  │
│     - VQE/QAOA iterative algorithms                         │
│     - Dedicated access to backend                           │
│     - Sequential jobs with shared context                   │
│     - Must close the room when done!                        │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

> **Memory Trick**: "Job = Just One, Batch = Bundle, Session = Sequential"

---

## Setup

In [ ]:
# Setup - Using FakeBackend for local testing
from qiskit import QuantumCircuit
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.primitives import StatevectorSampler

# Create a simple test circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Use local simulator for demonstration
sampler = StatevectorSampler()
job = sampler.run([qc], shots=1000)

# Job completes immediately with local simulator
result = job.result()
counts = result[0].data.meas.get_counts()

print("Job Lifecycle Demonstration:")
print(f"Circuit: Bell state with measurement")
print(f"Results: {counts}")

print("\nKey Job States (on real hardware):")
print("  - QUEUED: Waiting in queue")
print("  - RUNNING: Executing on hardware")
print("  - DONE: Completed successfully")
print("  - ERROR: Failed execution")
print("  - CANCELLED: User cancelled")

## Part 1: Job Lifecycle

### job.result() vs job.status()

```python
# job.result() - BLOCKING
result = job.result()  # Waits until job completes
# Returns: PrimitiveResult object

# job.status() - NON-BLOCKING  
status = job.status()  # Returns immediately
# Returns: JobStatus enum (QUEUED, RUNNING, DONE, etc.)
```

**EXAM TIP**: `result()` blocks, `status()` does not!

## Part 2: Job Monitoring

In [ ]:
# Job Monitoring - Local simulation example
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

sampler = StatevectorSampler()
job = sampler.run([qc], shots=1000)

# Local jobs complete immediately, but show the pattern
result = job.result()
print("Job Monitoring (local simulation):")
print(f"Job completed with results: {result[0].data.c.get_counts()}")

# For real hardware, you would use:
# job.status()         # Check status (non-blocking)
# job.queue_position() # Check queue position
# job.job_id()         # Get job ID for retrieval
# job.cancel()         # Cancel if needed

print("\n✓ job_id() for tracking")
print("✓ queue_position() shows place in line")
print("✓ cancel() stops execution")

## Part 3: Job Retrieval

In [ ]:
# Job Retrieval Pattern (requires real hardware)
print("Job Retrieval Pattern:")
print("=" * 50)

# For real hardware job retrieval:
# from qiskit_ibm_runtime import QiskitRuntimeService
# 
# # Save job ID when submitting
# job_id = job.job_id()
# 
# # Later: retrieve job
# service = QiskitRuntimeService()
# retrieved_job = service.job(job_id)
# result = retrieved_job.result()

# Demonstrate with local simulation
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

sampler = StatevectorSampler()
job = sampler.run([qc], shots=1000)
result = job.result()

print(f"Local simulation result: {result[0].data.c.get_counts()}")
print("\n✓ service.job(job_id) retrieves job on real hardware")
print("✓ Works even after program closes")
print("✓ Job history persists in IBM Cloud")

## Part 4: Sessions (EXAM CRITICAL!)

### Why Sessions?

Sessions provide **reserved, sequential access** to quantum hardware:

```
Without Session:                  With Session:
Job1 → Queue → Run               Job1 → Reserved slot → Run
Job2 → Queue → Run               Job2 → Same slot → Run immediately
Job3 → Queue → Run               Job3 → Same slot → Run immediately
(Each job waits in queue!)        (Reserved access = no queue!)
```

### Session Benefits
- **Reduced queue time**: Reserved slot for multiple jobs
- **Iterative algorithms**: Perfect for VQE/QAOA where Job N depends on Job N-1
- **Cost-effective**: Pay for session time, not per-job overhead

In [ ]:
# Session Pattern - Using FakeBackend for transpilation demo
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.primitives import StatevectorSampler

# FakeBackend for realistic transpilation
backend = FakeManilaV2()

# Create parameterized circuit (VQE-like)
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Transpile for realistic hardware constraints
transpiled = transpile(qc, backend=backend, optimization_level=3)

# Run with local sampler
sampler = StatevectorSampler()
job = sampler.run([transpiled], shots=1000)
result = job.result()

print("Session Pattern Demo (using FakeBackend topology):")
print(f"Transpiled circuit depth: {transpiled.depth()}")
print(f"Results: {result[0].data.c.get_counts()}")

print("\n--- For Real Hardware Sessions ---")
print("""
from qiskit_ibm_runtime import Session, SamplerV2

with Session(backend=backend) as session:
    sampler = SamplerV2(session=session)  # NOT backend=backend!
    
    for i in range(10):  # VQE iterations
        job = sampler.run([circuit])
        result = job.result()
        # Update circuit based on result
""")

print("\n🎯 EXAM TIP: Sampler(session=session) NOT Sampler(backend=backend)!")

### Session Benefits

| Feature | Single Jobs | Session |
|---------|-------------|----------|
| Queue priority | Normal | Higher |
| Resource reservation | No | Yes |
| Latency between jobs | High | Low |
| Use case | One-off | Iterative (VQE) |

**EXAM TIP**: Sessions recommended for 3+ related jobs!

### ⚠️ EXAM TRAP: Session vs Backend Parameter

```python
# CORRECT - Use session parameter
with Session(backend=backend) as session:
    sampler = Sampler(session=session)  # ✓

# WRONG - Don't use backend in session
with Session(backend=backend) as session:
    sampler = Sampler(backend=backend)  # ❌
```

## Part 5: Batch Execution

### What is Batch?

**Batch** runs multiple **independent** circuits in parallel (unlike Session which is sequential).

```python
from qiskit_ibm_runtime import Batch, SamplerV2 as Sampler

with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)
    
    # All circuits submitted together
    circuits = [qc1, qc2, qc3, qc4, qc5]
    job = sampler.run(circuits, shots=1024)
    
    # Results for all circuits
    results = job.result()
```

### Session vs Batch Decision

| Need | Use |
|------|-----|
| Sequential, dependent jobs (VQE) | **Session** |
| Parallel, independent circuits | **Batch** |
| Parameter sweeps | **Batch** |
| Iterative optimization | **Session** |

## Part 5: VQE Session Pattern

In [ ]:
# VQE-like Pattern Demo with Local Simulation
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.primitives import StatevectorSampler
import numpy as np

backend = FakeManilaV2()

# Simulate VQE optimization iterations
print("VQE-like Pattern Demo (local simulation):")
print("=" * 50)

sampler = StatevectorSampler()

# Simple parameterized circuit
for iteration in range(3):
    # Create circuit with varying rotation (simulates parameter update)
    theta = np.pi / (iteration + 2)
    qc = QuantumCircuit(2, 2)
    qc.ry(theta, 0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    # Transpile for FakeBackend topology
    transpiled = transpile(qc, backend=backend)
    
    # Run and get result
    job = sampler.run([transpiled], shots=500)
    result = job.result()
    counts = result[0].data.c.get_counts()
    
    print(f"Iteration {iteration + 1}: theta={theta:.3f}, counts={counts}")

print("\n✓ Session keeps backend reserved (on real hardware)")
print("✓ Minimal latency between iterations")
print("✓ Higher priority in queue")

## 📝 Practice Questions

### Question 1: Blocking Behavior

**Which method blocks until job completes?**

A) `job.status()`  
B) `job.queue_position()`  
C) `job.result()`  
D) `job.job_id()`

<details>
<summary>Answer</summary>

**C) `job.result()`**

```python
# Blocking (waits)
result = job.result()  # ✓ Blocks until DONE

# Non-blocking (immediate)
status = job.status()           # Returns immediately
position = job.queue_position() # Returns immediately
job_id = job.job_id()           # Returns immediately
```
</details>

---

### Question 2: Session Usage

**How to pass session to Sampler?**

A) `Sampler(backend=session)`  
B) `Sampler(session=session)`  
C) `Sampler(context=session)`  
D) `Sampler().set_session(session)`

<details>
<summary>Answer</summary>

**B) `Sampler(session=session)`**

```python
with Session(backend=backend) as session:
    sampler = Sampler(session=session)  # ✓ Correct
```

**CRITICAL**: Use `session` parameter, NOT `backend`!
</details>

---

### Question 3: When to Use Sessions

**When are Sessions recommended?**

A) Single circuit execution  
B) Iterative algorithms (VQE, QAOA)  
C) Only for simulators  
D) Never - deprecated

<details>
<summary>Answer</summary>

**B) Iterative algorithms (VQE, QAOA)**

Sessions best for:
- VQE optimization loops
- QAOA parameter sweeps
- Multiple related jobs (3+)

Benefits:
- Higher queue priority
- Reserved resources
- Lower latency between jobs
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **Job Lifecycle**
   - QUEUED → RUNNING → DONE
   - `job.result()` blocks
   - `job.status()` non-blocking

2. **Session Pattern**
   - `with Session(backend=backend) as session:`
   - `Sampler(session=session)` NOT `backend=backend`
   - Auto-closes after context exit

3. **Benefits**
   - Sessions: Higher priority, reserved resources
   - Recommended for 3+ related jobs
   - Critical for VQE/QAOA

### Critical Exam Facts

- ✅ `job.result()` BLOCKS until done
- ✅ `job.status()` returns immediately
- ✅ Session uses `session` parameter not `backend`
- ✅ Sessions recommended for iterative algorithms
- ✅ `job.job_id()` for retrieval
- ✅ Context manager auto-closes session

### Mnemonic

🧠 **"result() Waits, status() Checks, Session Groups!"**

**Next**: Options configuration!